<h2>KOSMOS slitmask acqusition

This notebook demonstrates basic idea of doing quick acquisition for KOSMOS slitmasks. Basic idea is to locate slits from flat field image and display location of alignment holes and slits. User selects which stars on the image correspond to the alignment holes. Code associates these with nearest alignment holes and calculates offset and rotation to apply to put stars in the holes.

In [1]:
from pyvista import imred, tv, stars, slitmask, image
import pdb
import copy
import numpy as np
import matplotlib.pyplot as plt
import os
from astropy.table import vstack

In [2]:
# you may need/want to use qt or osx in the next line
%matplotlib tk 
t=tv.TV()

In [4]:
red=imred.Reducer('KOSMOS',dir='/home/holtz/raw/apo/UT221221_cleaned',verbose=False)                                                         

In [5]:
red.log()

FILE,DATE-OBS,OBJNAME,RA,DEC,EXPTIME
bytes43,bytes26,bytes16,bytes16,bytes16,bytes16
bias.0001.fits,2022-12-21T07:03:19.199073,MUSSES2022D,7:44:26.48,44:17:03.60,0
bias.0002.fits,2022-12-21T07:04:12.915674,MUSSES2022D,7:44:26.48,44:17:03.60,0
bias.0003.fits,2022-12-21T07:05:07.210469,MUSSES2022D,7:44:26.48,44:17:03.60,0
bias.0004.fits,2022-12-21T07:06:00.729529,MUSSES2022D,7:44:26.48,44:17:03.60,0
bias.0005.fits,2022-12-21T07:06:54.510369,MUSSES2022D,7:44:26.48,44:17:03.60,0
mask63_arc_Ne_red_0p5s.0017.fits,2022-12-21T09:04:31.086764,eFEDS197p1,8:36:57.96,3:01:04.69,0.5
mask63_arc_Ne_red_0p5s.0018.fits,2022-12-21T09:05:25.327441,eFEDS197p1,8:36:57.96,3:01:04.69,0.5
mask63_arc_Ne_red_0p5s.0019.fits,2022-12-21T09:06:19.582805,eFEDS197p1,8:36:57.96,3:01:04.69,0.5
mask63_arc_Ar_red_0p5s.0020.fits,2022-12-21T09:10:43.446486,eFEDS197p1,8:36:57.96,3:01:04.69,0.5


Read in image with mask in, and image without mask

In [6]:
mask=red.reduce(51)
t.clear()
t.tv(mask,max=3000)

INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


In [7]:
nomask=red.reduce(52)
t.tv(nomask,max=30000)

INFO: array provided for uncertainty; assuming it is a StdDevUncertainty. [astropy.nddata.ccddata]


Do CR rejection on mask image to avoid spurious detections

In [8]:
maskcr=red.crrej(mask,crbox=[5,5])

Use findholes() to identify the holes in the mask. May need to adjust thresh= for brightness threshold, and n= for number of clusters to detect.

In [9]:
holes=slitmask.findholes(maskcr.data,thresh=1500,n=29)
t.clear()
t.tv(maskcr)
stars.mark(t,holes,color='b',exit=True,rad=10)

y,x
float64,float64
1847.4569536423842,1388.3741721854306
2194.5,728.0
1864.5,616.0
1829.5,1008.0
2627.5353535353534,489.050505050505
1274.6354166666667,799.4965277777778
1998.4494773519164,1790.212543554007
2221.3120567375886,910.5177304964539
1621.7963636363636,1104.3454545454545


Get design from KMS file and see if we have correct numbers of slits/holes identified

In [10]:
kmsfile='eFEDS197_pointing2.1.kms'
targets = slitmask.read_kms(kmsfile,sort='YMM')
print(len(targets),len(holes))
targets

15 15


ID,NAME,SHAPE,WID,LEN,ROT,ALPHA,DELTA,WIDMM,LENMM,XMM,YMM
str7,str5,str8,float64,float64,float64,float64,float64,float64,float64,float64,float64
TARG114,NN,STRAIGHT,4.0,4.0,0.0,83654.162,30638.76,0.683,0.683,-6.272,-33.306
TARG111,NN,STRAIGHT,4.0,4.0,0.0,83645.269,30628.86,0.683,0.683,16.462,-31.616
TARG109,Obj9,STRAIGHT,2.0,10.0,0.0,83702.29,30538.62,0.341,1.707,-27.05,-23.042
TARG101,Obj1,STRAIGHT,2.0,10.0,0.0,83649.106,30504.92,0.341,1.707,6.652,-17.29
TARG103,Obj3,STRAIGHT,2.0,10.0,0.0,83654.799,30436.19,0.341,1.707,-7.901,-12.387
TARG102,Obj2,STRAIGHT,2.0,10.0,0.0,83638.93,30417.69,0.341,1.707,32.666,-9.23
TARG104,Obj4,STRAIGHT,2.0,10.0,0.0,83655.258,30349.0,0.341,1.707,-9.075,-4.333
TARG105,Obj5,STRAIGHT,2.0,10.0,0.0,83648.518,30323.98,0.341,1.707,8.155,-0.063
TARG106,Obj6,STRAIGHT,2.0,10.0,0.0,83644.95,30258.99,0.341,1.707,17.277,4.202


Now sort the identified holes from left to right and match them
with the correspondingly sorted design holes to identify which ones are alignment holes.

In [11]:
# sort holes by x coordinate
holes.sort('x')

# loop through holes and targets to find alignment holes
align=[]
for i,(hole,target) in enumerate(zip(holes,targets)) :
    #print(hole['x'],target['YMM'],target['NAME'])
    if target['NAME'] == 'NN' : align.append(i)

# Display the alignment holes
stars.mark(t,holes[align],color='r',rad=20,exit=True)

#make table of the measured alignment hole positions
h=holes[align]
h

y,x
float64,float64
2158.0,255.0
1643.0,292.5
1255.0,1179.0
1992.5185185185185,1257.530864197531
2451.4347826086955,1525.7782608695652


Now display image and mark stars that corresonds to the alignment holes in the mask. Using the 'c' key, you just need to mark nearby to the star, and the code will centroid using that as a starting guess.

In [12]:
t.tv(nomask,max=30000)
s=stars.mark(t,rad=7)

# sort these by x coorindate to match order of alignment holes
s.sort('x')
s

Hit c near desired star(s) to get centroid position
    i to use integer position of cursor
    n to get ID of nearest star
    q or e to quit


id,x,y,MJD,EXPTIME,FILTER,AIRMASS
int32,float32,float32,float64,float32,bytes1,float32
1,221.95,2205.28,59934.466355,30.0,,1.244
2,260.76,1691.17,59934.466355,30.0,,1.244
4,1146.17,1306.40,59934.466355,30.0,,1.244
3,1223.02,2043.20,59934.466355,30.0,,1.244
5,1489.37,2506.04,59934.466355,30.0,,1.244


Now use fit() to get the rotation and translation. Output the distance between the measured object and the fit one to ensure that the identification is correct and the solution is reasonable: if mod-obs gives residuals larger than a couple of pixels, you may have the wrong association and solution!

In [19]:
slitmask.fit(h,s,fitscale=False)

Euclidean Transform: 
  Rotation (degrees) :  0.22610756795915388
  Translation (arcsec)  [-8.84721991 13.13450986]
 X       Y       mod-obs 
  -769.0   110.0    1.77
  -731.5  -405.0    0.96
   155.0  -793.0    1.67
   233.5   -55.5    1.23
   501.8   403.4    1.80
  Rotation (degrees) :  0.22610756795915388
  Translation (arcsec)  [-8.84721991 13.13450986]


(0.22610756795915388, array([-8.84721991, 13.13450986]))

invalid command name "47506080237248delayed_destroy"
    while executing
"47506080237248delayed_destroy"
    ("after" script)


In [14]:
help(slitmask.fit)

Help on function fit in module pyvista.slitmask:

fit(holes, locstars, ordered=False, fitscale=False)
    Given tables of hole and star locations, find offset and rotation
    
    Parameters
    ----------
    holes : astropy Table
            table of hole positions
    stars : astropy Table
            table of star positions
    ordered : bool, default=False
            if False, match holes to stars by closest distance.
            if True, stars and holes should match by table order
              (and tables should have the same number of entries)
    fitscale : bool, default=False
            if True, solve for scale as well as rotation and translation
    
    Returns
    -------
    
    rotation (degrees)
    translation (arcsec, assuming .258 arcsec/pix)
    scale (if fitscale=True)

